# `auto_arima`

Pmdarima bring R's [`auto.arima`](https://www.rdocumentation.org/packages/forecast/versions/7.3/topics/auto.arima) functionality to Python by wrapping statsmodel [`ARIMA`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/arima_model.py) and [`SARIMAX`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/statespace/sarimax.py) models into a singular scikit-learn-esque estimator ([`pmdarima.arima.ARIMA`](https://github.com/alkaline-ml/pmdarima/blob/master/pmdarima/arima/arima.py)) and adding several layers of degree and seasonal differencing tests to identify the optimal model parameters.

__Pmdarima ARIMA models:__

  - Are fully picklable for easy persistence and model deployment
  - Can handle seasonal terms (unlike statsmodels ARIMAs)
  - Follow sklearn model fit/predict conventions

In [1]:
import numpy as np
import pandas as pd
import pmdarima as pm

print('numpy version: %r' % np.__version__)
print('pandas version: %r' % pd.__version__)
print('pmdarima version: %r' % pm.__version__)

numpy version: '1.21.5'
pandas version: '1.3.5'
pmdarima version: '1.8.5'


We'll start by defining an array of data from an R time-series, `wineind`:

```r
> forecast::wineind
       Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov   Dec
1980 15136 16733 20016 17708 18019 19227 22893 23739 21133 22591 26786 29740
1981 15028 17977 20008 21354 19498 22125 25817 28779 20960 22254 27392 29945
1982 16933 17892 20533 23569 22417 22084 26580 27454 24081 23451 28991 31386
1983 16896 20045 23471 21747 25621 23859 25500 30998 24475 23145 29701 34365
1984 17556 22077 25702 22214 26886 23191 27831 35406 23195 25110 30009 36242
1985 18450 21845 26488 22394 28057 25451 24872 33424 24052 28449 33533 37351
1986 19969 21701 26249 24493 24603 26485 30723 34569 26689 26157 32064 38870
1987 21337 19419 23166 28286 24570 24001 33151 24878 26804 28967 33311 40226
1988 20504 23060 23562 27562 23940 24584 34303 25517 23494 29095 32903 34379
1989 16991 21109 23740 25552 21752 20294 29009 25500 24166 26960 31222 38641
1990 14672 17543 25453 32683 22449 22316 27595 25451 25421 25288 32568 35110
1991 16052 22146 21198 19543 22084 23816 29961 26773 26635 26972 30207 38687
1992 16974 21697 24179 23757 25013 24019 30345 24488 25156 25650 30923 37240
1993 17466 19463 24352 26805 25236 24735 29356 31234 22724 28496 32857 37198
1994 13652 22784 23565 26323 23779 27549 29660 23356
```

Note that the frequency of the data is 12:

```r
> frequency(forecast::wineind)
[1] 12
```

In [2]:
from pmdarima.datasets import load_wineind

# this is a dataset from R
wineind = load_wineind().astype(np.float64)

In [3]:
type(wineind)

numpy.ndarray

In [4]:
wineind.shape

(176,)

In [5]:
pd.DataFrame(wineind).head()

,0
0,15136.0
1,16733.0
2,20016.0
3,17708.0
4,18019.0


In [6]:
pd.DataFrame(wineind).isnull().sum()

0    0
dtype: int64

## Fitting an ARIMA

We will first fit a seasonal ARIMA. Note that you do not need to call `auto_arima` in order to fit a model&mdash;if you know the order and seasonality of your data, you can simply fit an ARIMA with the defined hyper-parameters:

In [7]:
from pmdarima.arima import ARIMA

model = ARIMA(order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
seasonal_order = model.fit(y=wineind)
seasonal_order

ARIMA(order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))

Also note that your data does not have to exhibit seasonality to work with an ARIMA. We could fit an ARIMA against the same data with no seasonal terms whatsoever (but it is unlikely that it will perform better; quite the opposite, likely).

In [8]:
model = ARIMA(order=(1, 1, 1), seasonal_order=None)
try:
    model.fit(y=wineind)
    print('seasonal_order: ', model.get_params['seasonal_order'])
except TypeError as e:
    print(e)

'NoneType' object is not subscriptable


## Finding the optimal model hyper-parameters using `auto_arima`:

If you are unsure (as is common) of the best parameters for your model, let `auto_arima` figure it out for you. `auto_arima` is similar to an ARIMA-specific grid search, but (by default) uses a more intelligent `stepwise` algorithm laid out in a paper by Hyndman and Khandakar (2008). If `stepwise` is False, the models will be fit similar to a gridsearch. Note that it is possible for `auto_arima` not to find a model that will converge; if this is the case, it will raise a `ValueError`.

### Fitting a stepwise search:

In [9]:
# fitting a stepwise model:
stepwise_fit = pm.auto_arima(wineind, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True, d=1, D=1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

stepwise_fit.summary()

Performing stepwise search to minimize aic


//anaconda3/envs/py3/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,1,1)(0,1,1)[12]             : AIC=3066.492, Time=0.34 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=3131.408, Time=0.03 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=3097.884, Time=0.12 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=3066.329, Time=0.13 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=3089.456, Time=0.04 sec


//anaconda3/envs/py3/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(0,1,1)(1,1,1)[12]             : AIC=3067.457, Time=0.20 sec
 ARIMA(0,1,1)(0,1,2)[12]             : AIC=3067.481, Time=0.34 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=3071.631, Time=0.11 sec


...

//anaconda3/envs/py3/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,1,2)(0,1,1)[12]             : AIC=3066.424, Time=0.36 sec
 ARIMA(0,1,3)(0,1,1)[12]             : AIC=3066.351, Time=0.34 sec


//anaconda3/envs/py3/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,1,3)(0,1,1)[12]             : AIC=3068.295, Time=0.64 sec
 ARIMA(0,1,2)(0,1,1)[12] intercept   : AIC=3066.787, Time=0.25 sec

Best model:  ARIMA(0,1,2)(0,1,1)[12]          
Total fit time: 7.783 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  176
Model:             SARIMAX(0, 1, 2)x(0, 1, [1], 12)   Log Likelihood               -1528.766
Date:                              Thu, 11 May 2023   AIC                           3065.533
Time:                                      16:50:56   BIC                           3077.908
Sample:                                           0   HQIC                          3070.557
                                              - 176                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.5756      0.041    -13.952      0.000      -0.656      -0.495
ma.L2         -0.1065      0.048     -2.224      0.026      -0.200      -0.013
ma.S.L12      -0.3848      0.054     -7.156      0.000      -0.490      -0.279
sigma2      7.866e+06   7.01e+05     11.228      0.000    6.49e+06    9.24e+06
===================================================================================
Ljung-Box (L1) (Q):                   2.84   Jarque-Bera (JB):                18.05
Prob(Q):                              0.09   Prob(JB):                         0.00
Heteroskedasticity (H):               1.17   Skew:                            -0.55
Prob(H) (two-sided):                  0.56   Kurtosis:                         4.21
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Fitting a random search

If you don't want to use the `stepwise` search, `auto_arima` can fit a random search by enabling `random=True`. If your random search returns too many invalid (nan) models, you might try increasing `n_fits` or making it an exhaustive search (`stepwise=False, random=False`).

In [10]:
rs_fit = pm.auto_arima(wineind, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                       start_P=0, seasonal=True, d=1, D=1, trace=True,
                       n_jobs=-1,  # We can run this in parallel by controlling this option
                       error_action='ignore',  # don't want to know if an order does not work
                       suppress_warnings=True,  # don't want convergence warnings
                       stepwise=False, random=True, random_state=42,  # we can fit a random search (not exhaustive)
                       n_fits=25)

rs_fit.summary()

//anaconda3/envs/py3/lib/python3.7/site-packages/pmdarima/arima/_auto_solvers.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gen = random_state.permutation(list(gen))[:n_fits]



Best model:  ARIMA(0,1,2)(1,1,1)[12]          
Total fit time: 22.730 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  176
Model:             SARIMAX(0, 1, 2)x(1, 1, [1], 12)   Log Likelihood               -1528.119
Date:                              Thu, 11 May 2023   AIC                           3066.239
Time:                                      16:51:19   BIC                           3081.708
Sample:                                           0   HQIC                          3072.519
                                              - 176                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.5586      0.043    -13.004      0.000      -0.643      -0.474
ma.L2         -0.1147      0.048     -2.379      0.017      -0.209      -0.020
ar.S.L12       0.2244      0.158      1.417      0.156      -0.086       0.535
ma.S.L12      -0.5922      0.157     -3.765      0.000      -0.901      -0.284
sigma2      7.866e+06   7.74e+05     10.166      0.000    6.35e+06    9.38e+06
===================================================================================
Ljung-Box (L1) (Q):                   3.07   Jarque-Bera (JB):                14.97
Prob(Q):                              0.08   Prob(JB):                         0.00
Heteroskedasticity (H):               1.19   Skew:                            -0.49
Prob(H) (two-sided):                  0.53   Kurtosis:                         4.12
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Inspecting goodness of fit

We can look at how well the model fits in-sample data:

In [19]:
from bokeh.plotting import figure, show, output_notebook
import pandas as pd

# init bokeh
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(truth.index, truth.values, color=c1, legend_label='Observed')
    p.line(forecasts.index, forecasts.values, color=c2, legend_label='Forecasted')
    
    return p

Loading BokehJS ...

In [20]:
in_sample_preds = stepwise_fit.predict_in_sample()
in_sample_preds[:10]

array([    0.        ,  9824.81821   , 12867.12617246, 16081.18015851,
       16446.13596489, 17110.88209357, 18038.82737534, 20257.17107204,
       21637.21204613, 21173.79437422])

In [21]:
show(plot_arima(wineind, in_sample_preds, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

## Predicting future values

After your model is fit, you can forecast future values using the `predict` function, just like in sci-kit learn:

In [22]:
next_25 = stepwise_fit.predict(n_periods=25)
next_25

array([24621.34759856, 29197.77533412, 34358.68997144, 12079.49380205,
       18865.29136135, 20827.98297551, 23122.49571555, 21226.64428737,
       23378.65225324, 26717.73986558, 22579.12927623, 19669.1990063 ,
       23819.46403643, 28395.40169327, 33556.31633059, 11277.1201612 ,
       18062.9177205 , 20025.60933466, 22320.1220747 , 20424.27064652,
       22576.27861239, 25915.36622473, 21776.75563539, 18866.82536545,
       23017.09039558])

In [23]:
# call the plotting func
show(plot_arima(wineind, next_25))

## Updating your model

ARIMAs create forecasts by using the latest observations. Over time, your forecasts will drift, and you'll need to update the model with the observed values. There are several solutions to this problem:

* Fit a new ARIMA with the new data added to your training sample
  - You can either re-use the order discovered in the `auto_arima` function, or re-run `auto_arima` altogether.
* Use the `update` method (__preferred__). This will allow your model to update its parameters by taking several more MLE steps on new observations (controlled by the `maxiter` arg) starting from the parameters it's already discovered. This approach will help you avoid over-fitting.

For this example, let's update our existing model with the `next_25` we just computed, as if they were actually observed values.

In [24]:
stepwise_fit.update(next_25, maxiter=10)  # take 10 more steps
stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  226
Model:             SARIMAX(0, 1, 2)x(0, 1, [1], 12)   Log Likelihood               -1968.260
Date:                              Thu, 11 May 2023   AIC                           3944.520
Time:                                      17:19:18   BIC                           3957.965
Sample:                                           0   HQIC                          3949.953
                                              - 226                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.5918      0.036    -16.296      0.000      -0.663      -0.521
ma.L2         -0.1067      0.041     -2.608      0.009      -0.187      -0.027
ma.S.L12      -0.3884      0.047     -8.298      0.000      -0.480      -0.297
sigma2      6.611e+06   4.85e+05     13.639      0.000    5.66e+06    7.56e+06
===================================================================================
Ljung-Box (L1) (Q):                   3.14   Jarque-Bera (JB):                77.75
Prob(Q):                              0.08   Prob(JB):                         0.00
Heteroskedasticity (H):               0.37   Skew:                            -0.73
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.57
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [25]:
updated_data = np.concatenate([wineind, next_25])

In [26]:
# visualize new forecasts
show(plot_arima(updated_data, stepwise_fit.predict(n_periods=10)))